In [2]:
import protobuf

ModuleNotFoundError: No module named 'protobuf'

In [ ]:
pip install protobuf

In [4]:
import random
import socket
import time

In [6]:
s = socket.socket()         # Create a socket object
host = 'localhost' # Get local machine name
port = 9082
s.bind((host, port))        # Bind to the port

print ('Starting server on', host, port)
print ('The Web server URL for this would be http://%s:%d/' % (host, port))

Starting server on localhost 9082
The Web server URL for this would be http://localhost:9082/


In [ ]:
s.listen(5) 
while True:
    # Establish connection with client.    
    c, (client_host, client_port) = s.accept()
    print('Got connection from', client_host, client_port)
    #c.send('Server Online\n') # This is invalid HTTP header
    c.recv(1000) # should receive request from client. (GET ....)
    
    c.send(('HTTP/1.0 200 OK\n').encode('utf-8'))
    c.send(('Content-Type: text/html\n').encode('utf-8'))
    c.send(('\n').encode('utf-8')) # header and body should be separated by additional newline
    c.send(("""
        <html>
        <body>
        <h1>Hello World</h1> this is my server!
        </body>
        </html>
    """).encode('utf-8')) # Use triple-quote string.
    c.close()

Got connection from 127.0.0.1 51053


In [10]:
#%%writefile server_protoc.py

import socket
from micro_pb2 import Microondas

HOST = 'localhost'
PORT = 6789
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen(1)
micro = Microondas()
conn, addr = s.accept()
data = conn.recv(4096)
print(data)
micro.ParseFromString(data)
print(micro.temperatura)
#conn.sendall(data_string)
conn.close()

Overwriting server_protoc.py


In [ ]:
import socket
import time
#from senstemp_pb2 import SensorTemp

HOST = 'localhost'
PORT = 6789
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen(5)
while True:
    
    conn, addr = s.accept()
    data = conn.recv(4096)
    #if data:
        #sensortemp = SensorTemp()
    print(data)
        #sensortemp.ParseFromString(data)
    print('Temperatura do ambiente: ', data.decode('utf-8'), ' graus Celsius')

#conn.sendall(data_string)
conn.close()


b'5'
Temperatura do ambiente:  5  graus Celsius


In [ ]:
import socket
import time
from senstemp_pb2 import SensorTemp

HOST = 'localhost'
PORT = 6789
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))
s.listen(1)
while True:
    sensortemp = SensorTemp()
    conn, addr = s.accept()
    data = conn.recv(4096)
    print(data)
    sensortemp.ParseFromString(data)
    print('Temperatura do ambiente: ',sensortemp.temperatura, ' graus Celsius')

#conn.sendall(data_string)
conn.close()


b'\r\x00\x00\x18B'
Temperatura do ambiente:  38.0  graus Celsius


In [ ]:
import threading
import socket
from senstemp_pb2 import SensorTemp
class Servidor():
    def __init__(self, host="localhost", port=1010):

        self.clientes = []

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind((str(host), int(port)))
        self.sock.listen(10)
        self.sock.setblocking(False)

        aceptar = threading.Thread(target=self.aceptarCon)
        procesar = threading.Thread(target=self.procesarCon)

        aceptar.daemon = True
        aceptar.start()

        procesar.daemon = True
        procesar.start()

        while True:
            msg = input('->')
            if msg == 's':
                self.sock.close()
                sys.exit()
            else:
                
                #pass
    
    def aceptarCon(self):
        print("aceptarCon iniciado")
        while True:
            try:
                conn, addr = self.sock.accept()
                conn.setblocking(False)
                self.clientes.append(conn)
                
            except:
                pass
    def procesarCon(self):
        print("ProcesarCon iniciado")
        while True:
            if len(self.clientes) > 0:
                for c in self.clientes:
                    try:
                        data = c.recv(1024)
                        if data:
                            sensortemp = SensorTemp()
                            sensortemp.ParseFromString(data)
                            print('Temperatura do ambiente: ',sensortemp.temperatura, ' graus Celsius')
                            #print(data.decode('utf-8'))
                            #self.msg_to_all(data,c)
                    except:
                        pass


s = Servidor()

aceptarCon iniciado
ProcesarCon iniciado
Temperatura do ambiente:  38.26956558227539  graus Celsius
Temperatura do ambiente:  37.90399169921875  graus Celsius
Temperatura do ambiente:  38.0  graus Celsius
Temperatura do ambiente:  38.535369873046875  graus Celsius
Temperatura do ambiente:  37.84685516357422  graus Celsius
Temperatura do ambiente:  37.318687438964844  graus Celsius
Temperatura do ambiente:  37.27128982543945  graus Celsius
Temperatura do ambiente:  37.5340461730957  graus Celsius
Temperatura do ambiente:  38.0  graus Celsius
Temperatura do ambiente:  38.0  graus Celsius
Temperatura do ambiente:  38.76791000366211  graus Celsius
Temperatura do ambiente:  38.630035400390625  graus Celsius
Temperatura do ambiente:  38.0  graus Celsius
Temperatura do ambiente:  38.0  graus Celsius
Temperatura do ambiente:  37.35400390625  graus Celsius
Temperatura do ambiente:  37.94759750366211  graus Celsius
Temperatura do ambiente:  37.62522506713867  graus Celsius
Temperatura do ambient

In [3]:
from sensores_pb2 import Sensor

In [ ]:
import sys
import threading
import socket
from sensores_pb2 import Sensor
class Servidor():
    status=0
    tmpcozinha = 0
    tmpquarto = 0
    def __init__(self, host="localhost", port=1510):

        self.clientes = []

        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.bind((str(host), int(port)))
        self.sock.listen(10)
        self.sock.setblocking(False)

        aceptar = threading.Thread(target=self.aceptarCon)
        procesar = threading.Thread(target=self.procesarCon)

        aceptar.daemon = True
        aceptar.start()

        procesar.daemon = True
        procesar.start()

        while True:
            msg = input('->')
            if msg == 's':
                self.sock.close()
                sys.exit()
            else:
                pass
    
    def aceptarCon(self):
        print("aceptarCon iniciado")
        while True:
            try:
                conn, addr = self.sock.accept()
                conn.setblocking(False)
                self.clientes.append(conn)
                conn.send(('HTTP/1.0 200 OK\n').encode('utf-8'))
                conn.send(('Content-Type: text/html\n').encode('utf-8'))
                conn.send(('\n').encode('utf-8')) # header and body should be separated by additional newline
                conn.send(("""
                    <html>
                        <body>
                            <h1>Trabalho Sistemas Distribuidos!</h1>
                            <h3>Temperatura cozinha: </h3>
                """).encode('utf-8'))
                mensagem = "<h4>{} graus Celsius</h4>".format(self.tmpcozinha)
                conn.send(mensagem.encode('utf-8'))
                mensagem2 = "<h3>Temperatura quarto: </h3><h4>{} graus Celsius</h4>".format(self.tmpquarto)
                conn.send(mensagem2.encode('utf-8'))
                mensagem3 = "<h3>Status da lampada: </h3><h4>{}</h4>".format(self.status)
                conn.send(mensagem3.encode('utf-8'))
                conn.send(("""
                        </body>
                    </html>
                """).encode('utf-8')) # Use triple-quote string.
            except:
                pass
    def procesarCon(self):
        print("ProcesarCon iniciado")
        while True:
            if len(self.clientes) > 0:
                for c in self.clientes:
                    try:
                        data = c.recv(1024)
                        if data:
                            #print(data)
                            sensor = Sensor()
                            sensor.ParseFromString(data)
                            if sensor.nome=='quarto':
                                self.tmpquarto = sensor.temperatura_quarto
                                print('Temperatura do quarto: ',sensor.temperatura_quarto, ' graus Celsius')
                                conn.send(("""
                                    <html>
                                        <body>
                                            <h1>Trabalho Sistemas Distribuidos!</h1>
                                            <h3>Temperatura cozinha: </h3>
                                """).encode('utf-8'))
                                mensagem = "<h4>{} graus Celsius</h4>".format(self.tmpcozinha)
                                conn.send(mensagem.encode('utf-8'))
                                mensagem2 = "<h3>Temperatura quarto: </h3><h4>{} graus Celsius</h4>".format(self.tmpquarto)
                                conn.send(mensagem2.encode('utf-8'))
                                mensagem3 = "<h3>Status da lampada: </h3><h4>{}</h4>".format(self.status)
                                conn.send(mensagem3.encode('utf-8'))
                                conn.send(("""
                                        </body>
                                    </html>
                                """).encode('utf-8')) # Use triple-quote string.
                            elif sensor.nome=='cozinha':
                                self.tmpcozinha = sensor.temperatura_cozinha
                                print('Temperatura do quarto: ',sensor.temperatura_quarto, ' graus Celsius')
                                conn.send(("""
                                    <html>
                                        <body>
                                            <h1>Trabalho Sistemas Distribuidos!</h1>
                                            <h3>Temperatura cozinha: </h3>
                                """).encode('utf-8'))
                                mensagem = "<h4>{} graus Celsius</h4>".format(self.tmpcozinha)
                                conn.send(mensagem.encode('utf-8'))
                                mensagem2 = "<h3>Temperatura quarto: </h3><h4>{} graus Celsius</h4>".format(self.tmpquarto)
                                conn.send(mensagem2.encode('utf-8'))
                                mensagem3 = "<h3>Status da lampada: </h3><h4>{}</h4>".format(self.status)
                                conn.send(mensagem3.encode('utf-8'))
                                conn.send(("""
                                        </body>
                                    </html>
                                """).encode('utf-8')) # Use triple-quote string.
                                print('Temperatura da cozinha: ',sensor.temperatura_cozinha, ' graus Celsius')
                            else:
                                status = sensor.status_lampada
                                if sensor.status_lampada == 1:
                                    print('Status da lampada: Ligada')
                                else:
                                    print('Status da lampada: Desligada')
                            #print(data.decode('utf-8'))
                            #self.msg_to_all(data,c)
                    except:
                        pass


s = Servidor()

aceptarCon iniciado
ProcesarCon iniciado
Temperatura do quarto:  37.37720489501953  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.0  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.09322738647461  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.69252014160156  graus Celsius
Temperatura do quarto:  37.525455474853516  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.00823974609375  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.0  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.36753845214844  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.103694915771484  graus Celsius
Temperatura do quarto:  38.739505767822266  graus Celsius
Temperatura do quarto:  0.0  graus Celsius
Temperatura do quarto:  38.0  graus Celsius
Temperatura do quarto:  

In [2]:
#!/usr/bin/python3 
# Importing the 'cgi' module 
import cgi 
s = socket.socket()         # Create a socket object
host = 'localhost' # Get local machine name
port = 1515
s.bind((host, port))        # Bind to the port

print ('Starting server on', host, port)
print ('The Web server URL for this would be http://%s:%d/' % (host, port))

print("Content-type: text/html\r\n\r\n") 
print("<html><body>") 
print("<h1> Hello Program! </h1>") 
# Using the inbuilt methods 
  
form = cgi.FieldStorage() 
if form.getvalue("name"): 
    name = form.getvalue("name") 
    print("<h1>Hello" +name+"! Thanks for using my script!</h1><br />") 
if form.getvalue("happy"): 
    print("<p> Yayy! I'm happy too! </p>") 
if form.getvalue("sad"): 
    print("<p> Oh no! Why are you sad? </p>") 
  
# Using HTML input and forms method 
print("<form method='post' action='hello2.py'>") 
print("<p>Name: <input type='text' name='name' /></p>") 
print("<input type='checkbox' name='happy' /> Happy") 
print("<input type='checkbox' name='sad' /> Sad") 
print("<input type='submit' value='Submit' />") 
print("</form") 
print("</body></html>") 

Content-type: text/html


<html><body>
<h1> Hello Program! </h1>
<form method='post' action='hello2.py'>
<p>Name: <input type='text' name='name' /></p>
<input type='checkbox' name='happy' /> Happy
<input type='checkbox' name='sad' /> Sad
<input type='submit' value='Submit' />
</form
</body></html>


In [3]:
pip install requests

In [1]:
import random
import socket
import time
import requests
#!/usr/bin/python3 
# Importing the 'cgi' module 
import cgi 
s = socket.socket()         # Create a socket object
host = 'localhost' # Get local machine name
port = 6969
s.bind((host, port))        # Bind to the port

print ('Starting server on', host, port)
print ('The Web server URL for this would be http://%s:%d/' % (host, port))

Starting server on localhost 6969
The Web server URL for this would be http://localhost:6969/


In [ ]:
from http.client import HTTPConnection
from http.server import CGIHTTPRequestHandler
s.listen(5)
while True:
    # Establish connection with client.   
    c, (client_host, client_port) = s.accept()
    print('Got connection from', client_host, client_port)
    #c.send('Server Online\n') # This is invalid HTTP header
    c.recv(1000) # should receive request from client. (GET ....)
    #r = requests.head('.', data ={'key':'value'}) 
    #print(r)
    #request = urllib2.Request('http://localhost:6969/')
    #request.get_method = lambda : 'HEAD'
    #conn = HTTPConnection('www.google.com')
    #conn.request('HEAD', '/http://localhost:6969/')
    #res = conn.getresponse()

    #print(res.status, res.reason)
    c.send(('HTTP/1.0 200 OK\n').encode('utf-8'))
    c.send(('Content-Type: text/html\n').encode('utf-8'))
    c.send(('\n').encode('utf-8')) # header and body should be separated by additional newline
    c.send(("""
        <html>
        <body>
        <form action="/cgi-bin/recebe.py" method="post">
            <input type="text" name="name"/>
            <input type='checkbox' name='happy' value = 1/> Happy
            <input type='checkbox' name='sad' value = 0/> Sad
            <input type='submit' value='Submit' />
        </form"
        </body>
        </html>
    """).encode('utf-8')) # Use triple-quote string.
    #r = requests.get('http://localhost:6969/')
    #print(r.json())
    c.close()

Got connection from 127.0.0.1 52232
Got connection from 127.0.0.1 52233
Got connection from 127.0.0.1 52234
Got connection from 127.0.0.1 52236
Got connection from 127.0.0.1 52240
Got connection from 127.0.0.1 52241
Got connection from 127.0.0.1 52242
Got connection from 127.0.0.1 52244
Got connection from 127.0.0.1 52248
Got connection from 127.0.0.1 52249
Got connection from 127.0.0.1 52251
Got connection from 127.0.0.1 52253


In [ ]:
import os, sys
from http.server import HTTPServer, CGIHTTPRequestHandler

webdir = 'localhost'

port = 9696

srvaddr

srvrobj = HTTPServer()